In [1]:
def get_input(day):
    with open('../inputs/input%d.txt' % day) as f:
        return f.read().strip()

In [ ]:
day = 1
input = get_input(day)

m = {'(': 1, ')': -1}

print(sum(m[x] for x in input))

s = 0
for i, x in enumerate(input):
    s += m[x]
    if s < 0:
        print(i+1)
        break

In [ ]:
day = 2
input = get_input(day)

s = 0
for line in input.split('\n'):
    [l, w, h] = map(int, line.split('x'))
    s += 2*l*w + 2*l*h + 2*w*h + min(l*w, l*h, w*h)
print(s)

s = 0
for line in input.split('\n'):
    [l, w, h] = map(int, line.split('x'))
    s += 2 * min(l+w, l+h, w+h) + l*w*h
print(s)

In [ ]:
day = 3
input = get_input(day)

m = {
    '^': lambda x,y: (x, y+1),
    'v': lambda x,y: (x, y-1),
    '>': lambda x,y: (x+1, y),
    '<': lambda x,y: (x-1, y),
}

p = (0,0)
s = {p}
for x in input:
    p = m[x](*p)
    s.add(p)
print(len(s))

p1 = (0,0)
p2 = (0,0)
s = {p1}
for x1,x2 in zip(input[::2], input[1::2]):
    p1 = m[x1](*p1)
    p2 = m[x2](*p2)
    s.add(p1)
    s.add(p2)
print(len(s))

In [ ]:
day = 4
input = get_input(day)

import hashlib
from itertools import count

def f(s):
    for i in count(1):
        if hashlib.md5((input + str(i)).encode('utf-8')).hexdigest().startswith(s):
            return i

print(f("00000"))

print(f("000000"))

In [ ]:
day = 5
input = get_input(day)

import re

def is_nice(s):
    return len(re.findall(r'[aeiou]', s)) > 2 \
           and re.search(r'(.)\1', s) \
           and not re.search(r'ab|cd|pq|xy', s)
        
print(sum(1 for x in input.split('\n') if is_nice(x)))

def is_nice2(s):
    return re.search(r'(..).*\1', s) \
           and re.search(r'(.).\1', s)
    
print(sum(1 for x in input.split('\n') if is_nice2(x)))

In [ ]:
day = 6
input = get_input(day)

import re

def run_command(grid, cmd_str, fun1, fun2, fun3):
    m = re.match(r'(toggle|turn on|turn off) (\d+),(\d+) through (\d+),(\d+)', line)
    cmd = m.group(1)
    x1,y1,x2,y2 = map(int, m.group(2, 3, 4, 5))
    f = fun1 if cmd == "toggle" else fun2 if cmd == "turn on" else fun3
    for x in range(x1, x2+1):
        for y in range(y1, y2+1):
            grid[x][y] = f(grid[x][y])

grid = [[0 for _ in range(1000)] for _ in range(1000)]
for line in input.split('\n'):
    run_command(grid, line,
                lambda x: x ^ 1,
                lambda x: 1,
                lambda x: 0)

print(sum(map(sum, grid)))

grid = [[0 for _ in range(1000)] for _ in range(1000)]
for line in input.split('\n'):
    run_command(grid, line,
                lambda x: x + 2,
                lambda x: x + 1,
                lambda x: max(0, x-1))

print(sum(map(sum, grid)))

In [ ]:
day = 7
input = get_input(day)

import re
from collections import deque

ops = {
    'NOT': '~',
    'AND': '&',
    'OR': '|',
    'LSHIFT': '<<',
    'RSHIFT': '>>',
}

def parse(line):
    line = re.sub(r'([a-z]+)', 'nodes["\\1"]', line)
    expr, var = line.split(' -> ')
    for op, val in ops.items():
        expr = expr.replace(op, val)
    return var, expr

def eval_instr(instr, variables):
    try:
        exec(instr, {}, variables)
        return True
    except KeyError:
        return False

def get_node_value(instrs, varname):
    variables = {'nodes': {}}
    while len(instrs) > 0:
        instr = instrs.popleft()
        if not eval_instr(instr, variables):
            instrs.append(instr)
    return variables['nodes'][varname]

instrs = deque('{} = {}'.format(*parse(line)) 
               for line in  input.split('\n'))
sol1 = get_node_value(instrs, 'a')
print(sol1)

instr_map = dict(parse(line)
                 for line in  input.split('\n'))
instr_map['nodes["b"]'] = sol1
instrs = deque('{} = {}'.format(*pair) 
               for pair in instr_map.items())
sol2 = get_node_value(instrs, 'a')
print(sol2)

In [116]:
# Alternate Day 7
day = 7
input = get_input(day)

import re
from collections import defaultdict
from functools import partial
from operator import inv, and_, or_, lshift, rshift

# Probably some deferred library out there, oh well
class Deferred(object):
    def __init__(self, val=None):
        self.callbacks = []
        self.val = val
    
    def add_callback(self, callback):
        self.callbacks.append(callback)
        if self.val is not None:
            callback(self.val)
    
    def callback(self, val):
        self.val = val
        for callback in self.callbacks:
            callback(self.val)

class Node(object):
    def __init__(self, a=None, op=None, b=None):
        self.a = None
        self.op = op
        self.b = None
        def callback(self, attr, val):
            setattr(self, attr, val)
        if a:
            a.add_callback(partial(callback, self, 'a'))
        if b:
            b.add_callback(partial(callback, self, 'b'))
        self.val = None
        self.dirty = False

    def get_val(self):
        self.dirty = True
        def get_param(v):
            return v if isinstance(v, int) else v.get_val()
        if self.val:
            pass
        elif not self.a and not self.op:
            self.val = get_param(self.b)
        elif not self.a:
            self.val = self.op(get_param(self.b))
        else:
            self.val = self.op(get_param(self.a),
                               get_param(self.b))
        return self.val
    
    def rebuild(self):
        self.val = None
        self.dirty = False
        if isinstance(self.a, Node) and self.a.dirty:
            self.a.rebuild()
        if isinstance(self.b, Node) and self.b.dirty:
            self.b.rebuild()
        

ops = {
    'NOT': inv,
    'AND': and_,
    'OR': or_,
    'LSHIFT': lshift,
    'RSHIFT': rshift,
}

def _int(n):
    try:
        return int(n)
    except (ValueError, TypeError):
        return n

def init_node_tree(input):
    m = defaultdict(Deferred)
    for line in input.split('\n'):
        r = re.match(r'(?:(?:(\S+) )?(\S+) )?(\S+) -> (\S+)', line)
        a, op, b, v = r.group(1, 2, 3, 4)
        a = _int(a)
        b = _int(b)
        a = Deferred(a) if isinstance(a, int) else m[a]
        b = Deferred(b) if isinstance(b, int) else m[b]
        m[v].callback(Node(a, ops.get(op), b))
    return m

nodes = init_node_tree(input)
ans1 = nodes['a'].val.get_val()
print(ans1)

nodes['a'].val.rebuild()
nodes['b'].callback(ans1)
print(nodes['a'].val.get_val())

46065
14134


In [ ]:
day = 8
input = get_input(day)

import re

s = 0
for line in input.split('\n'):
    s += len(line)
    s -= len(eval(line))
# Part 1
print(s)

s = 0
for line in input.split('\n'):
    s += len(re.escape(line))
    s += 2 # surrounding quotes
    s -= len(line)

# Part 2
print(s)